In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature


In [2]:
df = pd.read_csv("../data/raw/CarPrice_Assignment.csv")

In [3]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df = df.drop('car_ID',axis=1)

In [5]:
continous =[]
categorical =[]
cat_to_cont =[]
for col in df.columns:
    if df[col].dtypes != object:
        #print(col)
        continous.append(col)
    elif col == 'CarName':
        categorical.append(col)
     
    else:
        cat_to_cont.append(col)

In [6]:
target = "price"
for col in categorical:
    df[col] = df.groupby([col])[target].transform('mean')

In [7]:
for col in cat_to_cont:
    print(col, df[col].unique())

fueltype ['gas' 'diesel']
aspiration ['std' 'turbo']
doornumber ['two' 'four']
carbody ['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
drivewheel ['rwd' 'fwd' '4wd']
enginelocation ['front' 'rear']
enginetype ['dohc' 'ohcv' 'ohc' 'l' 'rotor' 'ohcf' 'dohcv']
cylindernumber ['four' 'six' 'five' 'three' 'twelve' 'two' 'eight']
fuelsystem ['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' '4bbl' 'idi' 'spdi']


In [8]:
df['fueltype'] = df['fueltype'].map({'gas':0,'diesel':1})
df['aspiration'] = df['aspiration'].map({'std':0,'turbo':1})
df['doornumber'] = df['doornumber'].map({'two':0,'four':1})
df['carbody'] = df['carbody'].map({'hardtop':0,'hatchback':1,'wagon':2,'sedan':3,'convertible':4})
df['drivewheel'] = df['drivewheel'].map({'rwd':0,'fwd':1,'4wd':2})
df['enginelocation'] = df['enginelocation'].map({'front':0,'rear':1})
df['enginetype']=df['enginetype'].map({'dohc':4, 'ohcv':3, 'ohc':1, 'l':0, 'rotor':6, 'ohcf':2, 'dohcv':5})
df['cylindernumber'] = df['cylindernumber'].map({'four':2, 'six':4, 'five':3, 'three':1, 'twelve':6, 'two':0, 'eight':5})
df['fuelsystem'] =  df['fuelsystem'].map({'mpfi':3, '2bbl':1, 'mfi':2, '1bbl':0, 'spfi':5, '4bbl':6, 'idi':7 ,'spdi':4})

In [9]:
df.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,13495.0,0,0,0,4,0,0,88.6,168.8,...,130,3,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,16500.0,0,0,0,4,0,0,88.6,168.8,...,130,3,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,16500.0,0,0,0,1,0,0,94.5,171.2,...,152,3,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,13950.0,0,0,1,3,1,0,99.8,176.6,...,109,3,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,17580.0,0,0,1,3,2,0,99.4,176.6,...,136,3,3.19,3.40,8.0,115,5500,18,22,17450.0


In [10]:
X1 = df.drop(['price'],axis=1)
y = df['price']

In [11]:
scaler = StandardScaler()

In [12]:
X = scaler.fit_transform(X1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [15]:
rf_regressor1 = RandomForestRegressor(random_state=42)


In [16]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Make sure the MLflow server is running
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/278357840985887951', creation_time=1744905586874, experiment_id='278357840985887951', last_update_time=1744905586874, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

In [17]:

# Hyperparameter search space
param_dist = {
    'n_estimators': np.arange(50, 301, 50),
    'max_depth': [None] + list(np.arange(5, 31, 5)),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 5)
}

In [23]:
# Start MLflow run
with mlflow.start_run() as run:
    # Randomized search CV
    random_search = RandomizedSearchCV(
        rf_regressor,
        param_distributions=param_dist,
        n_iter=20,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    y_pred = best_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Log hyperparameters and metrics
    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    # Create model signature
    signature = infer_signature(X_train, best_model.predict(X_train))

    # Log the model with signature and input example
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="random_forest_model",
        signature=signature,
        input_example=X_train[:5]
    )

    # Register the model
    model_uri = f"runs:/{run.info.run_id}/random_forest_model"
    model_name = "RandomForestRegressorModel"
    
    # Register the model in the MLflow Model Registry
    mlflow.register_model(model_uri=model_uri, name=model_name)

    # Save the best model to a pickle file for backup or FastAPI use
    joblib.dump(best_model, "random_forest_model.pkl")

    print("Model training, logging, registration, and local save completed.")
    

c:\Users\Manikandan\Desktop\production_1\car_price_pred\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Registered model 'RandomForestRegressorModel' already exists. Creating a new version of this model...
2025/04/17 22:24:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressorModel, version 2
Created version '2' of model 'RandomForestRegressorModel'.


Model training, logging, registration, and local save completed.
🏃 View run rogue-kite-414 at: http://127.0.0.1:5000/#/experiments/278357840985887951/runs/38c0eac0dcd54fec8990e2895f599313
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/278357840985887951
